In [1]:
# import libraries
import sys
import os
import h5py
import pandas as pd
import matplotlib.pylab as plt
from jupyterlab_h5web import H5Web
# import modules with the functionalities offered by CompositionSpace
from compositionspace.utils import get_file_size
from compositionspace.io import get_reconstructed_positions, get_iontypes, get_ranging_info
from compositionspace.preparation import ProcessPreparation
from compositionspace.segmentation import ProcessSegmentation
from compositionspace.clustering import ProcessClustering

In [2]:
# ! pip list
MY_PROCESSED_DATA_PATH = f"{os.getcwd()}"
print(f"Executing compositionspace in the following working directory: {os.getcwd()}")

Executing compositionspace in the following working directory: /home/kaiobach/Research/hu_hu_hu/sprint22/conda-compspace-step01/CompositionSpace


## Load reconstruction and ranging

In [3]:
workdir = "/home/kaiobach/Research/paraprobe-toolbox/teaching/example_analyses/iuc09_saksena"
workdir = "/home/kaiobach/Research/paraprobe-toolbox/teaching/example_analyses/usa_denton_smith"
simid = 1
RECONSTRUCTION_AND_RANGING = (f"{workdir}/..")
RECONSTRUCTION_AND_RANGING = (f"{workdir}/PARAPROBE.Transcoder.Results.SimID.1.nxs",
                              f"{workdir}/PARAPROBE.Ranger.Results.SimID.1.nxs")
config_file_path = f"{MY_PROCESSED_DATA_PATH}/experiment_params.yaml"
results_file_path = f"{MY_PROCESSED_DATA_PATH}/CompositionSpace.Results.{simid}.nxs"

In [4]:
# H5Web(RECONSTRUCTION_AND_RANGING[0])
# H5Web(RECONSTRUCTION_AND_RANGING[1])

In [5]:
(xyz_val, xyz_unit) = get_reconstructed_positions(RECONSTRUCTION_AND_RANGING[0])
ityp_info, nochrg_ityp_info, elements = get_ranging_info(RECONSTRUCTION_AND_RANGING[0], verbose=True)
(ityp_val, ityp_unit) = get_iontypes(RECONSTRUCTION_AND_RANGING[1])

Load reconstructed positions shape (945211, 3), type <class 'numpy.ndarray'>, dtype float32
26 iontypes distinguished:
	ion0, ('unknown iontype', 0)
	ion1, ('Si ++', 1)
	ion2, ('Si +', 2)
	ion3, ('Cr +', 3)
	ion4, ('Si +', 4)
	ion5, ('Si +', 5)
	ion6, ('Si ++', 6)
	ion7, ('Si ++', 7)
	ion8, ('Cr +', 8)
	ion9, ('Cr ++', 9)
	ion10, ('Cr ++', 10)
	ion11, ('Cu +', 11)
	ion12, ('Cu +', 12)
	ion13, ('C +', 13)
	ion14, ('C ++', 14)
	ion15, ('O +', 15)
	ion16, ('O +', 16)
	ion17, ('Cr O +', 17)
	ion18, ('Cr O +', 18)
	ion19, ('Cr O +', 19)
	ion20, ('Cr O ++', 20)
	ion21, ('Cr O ++', 21)
	ion22, ('Cr O ++', 22)
	ion23, ('Cr O O +', 23)
	ion24, ('Cr O O ++', 24)
	ion25, ('Cr Cr O ++', 25)
9 charge-agnostic iontypes distinguished:
	unknown iontype, [0]
	Si, [1, 2, 4, 5, 6, 7]
	Cr, [3, 8, 9, 10]
	Cu, [11, 12]
	C, [13, 14]
	O, [15, 16]
	Cr O, [17, 18, 19, 20, 21, 22]
	Cr O O, [23, 24]
	Cr Cr O, [25]
5 elements distinguished:
Cr
Cu
O
C
Si
Load ranged iontypes shape (945211,), type <class 'numpy.ndar

## Voxelize with rectangular transfer function without creating slices

In [6]:
voxelize = ProcessPreparation(config_file_path, results_file_path, verbose=True)
voxelize.init_ranging(ityp_info, elements)
voxelize.write_init_results()
voxelize.define_voxelization_grid(xyz_val)
voxelize.define_lookup_table(ityp_val)
voxelize.write_voxelization_grid_info()
voxelize.write_voxelization_results()

[[ 3.4028235e+38 -3.4028235e+38]
 [ 3.4028235e+38 -3.4028235e+38]
 [ 3.4028235e+38 -3.4028235e+38]]
shape (945211,)
dim 0
	np.min(xyz[:, axis_id]) -20.433578491210938 >>>> -23.0
	np.max(xyz[:, axis_id]) 19.706375122070312 >>>> 22.0
	self.aabb3d axis_id 0, [-23.  22.], extent 45
dim 1
	np.min(xyz[:, axis_id]) -17.857210159301758 >>>> -20.0
	np.max(xyz[:, axis_id]) 21.282995223999023 >>>> 24.0
	self.aabb3d axis_id 1, [-20.  24.], extent 44
dim 2
	np.min(xyz[:, axis_id]) -75.08065032958984 >>>> -78.0
	np.max(xyz[:, axis_id]) -0.020034153014421463 >>>> 2.0
	self.aabb3d axis_id 2, [-78.   2.], extent 80
np.max(self.voxel_identifier) 153609
np.prod(self.extent) 158400
ion1, ('Si ++', 1):
	Si
	1
	{'Si': 1}
ion2, ('Si +', 2):
	Si
	1
	{'Si': 1}
ion3, ('Cr +', 3):
	Cr
	1
	{'Cr': 1}
ion4, ('Si +', 4):
	Si
	1
	{'Si': 1}
ion5, ('Si +', 5):
	Si
	1
	{'Si': 1}
ion6, ('Si ++', 6):
	Si
	1
	{'Si': 1}
ion7, ('Si ++', 7):
	Si
	1
	{'Si': 1}
ion8, ('Cr +', 8):
	Cr
	1
	{'Cr': 1}
ion9, ('Cr ++', 9):
	Cr
	1
	{'

In [7]:
get_file_size(results_file_path)
# H5Web(results_file_path)

2.279 MiB


Voxelization based on element types not on iontypes.

## Segmentation PCA and IC minimization

In [8]:
segmentation = ProcessSegmentation(
    config_file_path,
    results_file_path,
    entry_id=1,
    verbose=True)
segmentation.perform_pca_and_write_results()
segmentation.perform_bics_minimization_and_write_results()

Composition matrix has 5 chemical classes
Composition matrix has 5 chemical classes
GaussianMixture ML analysis with n_cluster 1
GaussianMixture ML analysis with n_cluster 2
GaussianMixture ML analysis with n_cluster 3
GaussianMixture ML analysis with n_cluster 4
GaussianMixture ML analysis with n_cluster 5


In [9]:
get_file_size(results_file_path)
# H5Web(results_file_path)

2.473 MiB


<div class="alert alert-block alert-danger">
Discussion points:<br>
* Modify NXapm_composition_space<br>
* What to show how to show?<br>
* Number of cluster vs Number of clusters? wording...?<br>
* Why to run the gm several times, ones in the bics loop ones in get composition cluster files 
</div>

## DBScan clustering

In [10]:
clustering = ProcessClustering(
    config_file_path,
    results_file_path,
    entry_id=1,
    verbose=True)
clustering.run_and_write_results()

DBScan configuration: eps 3 nm, min_samples 5
['cluster_analysis0', 'cluster_analysis1', 'cluster_analysis2', 'cluster_analysis3', 'cluster_analysis4', 'sequence_index', 'summary']
cluster_analysis0
ic_run_id 0 >>>>
np.shape(all_vxl_pos) (158400, 3) list(set(phase_identifier) [0]
	Loop 0
	np.shape(trg_vxl_pos) (158400, 3)
	np.shape(trg_vxl_idx) (158400,)
	1
	type(db.labels_) <class 'numpy.ndarray'> dtype int64
[0]
cluster_analysis1
ic_run_id 1 >>>>
np.shape(all_vxl_pos) (158400, 3) list(set(phase_identifier) [0, 1]
	Loop 0
	np.shape(trg_vxl_pos) (48323, 3)
	np.shape(trg_vxl_idx) (48323,)
	1
	type(db.labels_) <class 'numpy.ndarray'> dtype int64
[0]
	Loop 1
	np.shape(trg_vxl_pos) (110077, 3)
	np.shape(trg_vxl_idx) (110077,)
	1
	type(db.labels_) <class 'numpy.ndarray'> dtype int64
[0]
cluster_analysis2
ic_run_id 2 >>>>
np.shape(all_vxl_pos) (158400, 3) list(set(phase_identifier) [0, 1, 2]
	Loop 0
	np.shape(trg_vxl_pos) (110077, 3)
	np.shape(trg_vxl_idx) (110077,)
	1
	type(db.labels_) <cla

In [11]:
get_file_size(results_file_path)
H5Web(results_file_path)

3.878 MiB


<jupyterlab_h5web.widget.H5Web object>

<div class="alert alert-block alert-danger">
Discussion points:<br>
- Tests are too specific, hardcoded file names<br>
- Readthedocs documentation needs to be updated<br>
- GM and ML models are variables collect over<br>
- Loading file formats from the community should use ifes-apt-tc-data-modeling library currently using paraprobe result<br>
- Ion handling should use ifes-apt-tc-data-modeling is not added as a dependencies and loading properly<br>
-  tests/experiment_params.json should be removed?<br>
- NeXus renaming<br>
- CompositionSpace by design does not distinguish charge states iontypes should be atomic decomposed<br>
- Why is the center of the voxel defined by the median position of the ions but not by the barycenter of the voxel (currently using voxel barycenter)<br>
- Ran 2, and even 0.5 discretization speed is comparable<br>
- Triple loop in preparation step should be replaced with more fancy numpy indexing code that I know is somewhere but I couldnt find quickly<br>
- Move test data out of this repository<br>
</div>

## Meshing

Test for now with the SiGe dataset.

In [ ]:
sige_file_path = "Output_DBSCAN_segmentation_phase_1.h5"
H5Web(sige_file_path)

In [ ]:
import h5py
import numpy as np
with h5py.File(sige_file_path, "r") as h5r:
    n_vxls = 0
    aabb3d = np.zeros((3, 2), np.float64)
    for dim in [0, 1, 2]:
        aabb3d[dim, :] = [np.finfo(np.float64).max, np.finfo(np.float64).min]
    # print(aabb3d)
    for key in h5r["1"].keys():
        for dim in [0, 1, 2]:
            mimx = (np.min(h5r["1"][key][:, dim]), np.max(h5r["1"][key][:, dim]))
            if mimx[0] <= aabb3d[dim, 0]:
                aabb3d[dim, 0] = mimx[0]
            if mimx[1] >= aabb3d[dim, 1]:
                aabb3d[dim, 1] = mimx[1]
            n_vxls += int(np.shape(h5r["1"][key])[0])
    print(aabb3d)
    print(n_vxls)
    # assume cubic vxl 2nm edge length

Assume that the data were discretized on the following rectangular grid with 2nm cubic voxel

In [ ]:
nx = int((88--96)/2)
ny = int((94--96)/2)
nz = int((0--222)/2)
grid = np.zeros((nx, ny, nz), np.uint32)
# that grid should intentionally be a cuboid to enable checking correct dimensions

def i_to_xyz(i):
    z = int(i / (nx * ny))
    rem = i - (nx * ny * z)
    y = int(rem / nx)
    x = rem - (y * nx)
    return (x, y, z)

with h5py.File(sige_file_path, "r") as h5r:
    for key in h5r["1"].keys():
        jds = np.asarray(h5r["1"][key][:, 3], np.uint32)
        for j in jds:
            x, y, z = i_to_xyz(j)
            grid[x, y, z] = int(key) + 1
        print(key)
print(np.shape(grid))
print(np.unique(grid))

In [ ]:
import h5py
with h5py.File("input.grid.nxs", "w") as h5w:
    h5w.create_dataset("/grid", compression="gzip", compression_opts=1, data=grid)

In [ ]:
H5Web("input.grid.nxs")